In [1]:
import os
import glob
import numpy as np

from pathlib import Path
from aicsimageio import AICSImage, readers
from blimp.preprocessing.illumination_correction import IlluminationCorrection

data_dir = "/srv/scratch/berrylab/z3532965/NikonSpinningDisk/240328/240325_siRNA_ARMC5_INTS8"
image_dir = os.path.join(data_dir,"20240328_172603_573/OME-TIFF-MIP")

ana_dir = "/srv/scratch/z3532965/src/publications/2024_ARMC5/hct116_IF"
illum_corr_file = "/srv/scratch/berrylab/z3532965/NikonSpinningDisk/240328/240325_siRNA_ARMC5_INTS8/ILLUMCORR/illumination_correction.pkl"

05-Apr-24 21:52:40 - bfio.backends - WARNING  - Java backend is not available. This could be due to a missing dependency (jpype).


# Apply illumination correction

Load illumination correction object

In [2]:
illumination_correction = IlluminationCorrection(from_file=illum_corr_file)

## Save examples for training nuclear segmentation model

In [3]:
training_dir = Path(image_dir).parent / 'TRAINING'
if not training_dir.exists(): 
    training_dir.mkdir()

In [4]:
example_image_files = glob.glob(str(Path(image_dir) / "*003.ome.tiff"))

In [5]:
for f in example_image_files[0:10]:
    aics_image = AICSImage(Path(image_dir) / Path(f).name, reader=readers.ome_tiff_reader.OmeTiffReader)
    aics_image_corrected = illumination_correction.correct(aics_image)
    # crop 500 pixels from left edge and save DAPI (C=1)
    DAPI = AICSImage(aics_image_corrected.get_image_data('YX',Z=0,C=1,T=0)[np.newaxis,np.newaxis,np.newaxis,:,500:])
    DAPI.save(str(training_dir / Path(f).name))

In [6]:
training_dir

PosixPath('/srv/scratch/berrylab/z3532965/NikonSpinningDisk/240328/240325_siRNA_ARMC5_INTS8/20240328_172603_573/TRAINING')

## Save examples for manual background measurements

In [7]:
examples_dir = Path(image_dir).parent / 'EXAMPLES'
if not examples_dir.exists(): 
    examples_dir.mkdir()

In [8]:
for f in example_image_files[1:4]:
    aics_image = AICSImage(Path(image_dir) / Path(f).name, reader=readers.ome_tiff_reader.OmeTiffReader)
    aics_image_corrected = illumination_correction.correct(aics_image)
    aics_image_corrected.save(examples_dir / Path(f).name)

In [9]:
examples_dir

PosixPath('/srv/scratch/berrylab/z3532965/NikonSpinningDisk/240328/240325_siRNA_ARMC5_INTS8/20240328_172603_573/EXAMPLES')